Esse notebook tem por objetivo fazer a relação de empresas que importam somente uma marca, facilitando a relação importador-

In [1]:
# Importing the modules needed
import sys

import pandas as pd

sys.path.append("../src/")

from src.data.dremio_utils import *
# Data Handling
from dotenv import dotenv_values 

import numpy as np
from tqdm import tqdm
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, TimestampType

# import pyspark.pandas as pd

In [46]:
spark = SparkSession.builder.master("local[1]").appName("attributes_dict").getOrCreate()
spark

In [47]:
config = dotenv_values(".env")
bds = BaseDremioService(config)

## 1. Getting Merged Data

# leitura do dataset histórico

In [48]:
historic_data = spark.read.parquet("../data/processed/average_unity_price_historic.parquet")
# Agrupar por semestre
historic_data.printSchema()

root
 |-- id_pais_origem: string (nullable = true)
 |-- ncm: string (nullable = true)
 |-- importador_uf: string (nullable = true)
 |-- importador_municipio: string (nullable = true)
 |-- urf: string (nullable = true)
 |-- anomes: string (nullable = true)
 |-- avg_valor_item: double (nullable = true)
 |-- semestre: long (nullable = true)
 |-- ano: long (nullable = true)
 |-- ano_semestre: long (nullable = true)
 |-- __index_level_0__: long (nullable = true)


# Completar com os meses futuros

In [49]:
spark.read.parquet("../data/processed/trend_values/trend_lines.parquet")

DataFrame[ano: int, semestre: bigint, ano_semestre: bigint, avg_valor_item: double, ncm: string, id_pais_origem: string]

In [50]:
# Creathe the date template to be crossed
template_data = [
    {"ano": 2018, "semestre": 1}, {"ano": 2018, "semestre": 2},
    {"ano": 2019, "semestre": 1}, {"ano": 2019, "semestre": 2},
    {"ano": 2020, "semestre": 1}, {"ano": 2020, "semestre": 2},
    {"ano": 2021, "semestre": 1}, {"ano": 2021, "semestre": 2},
    {"ano": 2022, "semestre": 1}, {"ano": 2022, "semestre": 2},
    {"ano": 2023, "semestre": 1}, {"ano": 2023, "semestre": 2}, 
    {"ano": 2024, "semestre": 1}, {"ano": 2024, "semestre": 2}]

template_schema = StructType([StructField("ano", IntegerType()),
                              StructField("semestre", IntegerType())])

data_template = spark.createDataFrame(data=template_data, schema=template_schema)

In [51]:
unique_combinations = historic_data.select(
    historic_data.ncm,
    historic_data.id_pais_origem,
    historic_data.importador_municipio,
    historic_data.importador_uf,
    historic_data.urf
).distinct()

In [52]:
unique_combinations.printSchema()

root
 |-- ncm: string (nullable = true)
 |-- id_pais_origem: string (nullable = true)
 |-- importador_municipio: string (nullable = true)
 |-- importador_uf: string (nullable = true)
 |-- urf: string (nullable = true)


In [53]:
data_2b_filled = unique_combinations.crossJoin(data_template)
data_2b_filled = data_2b_filled.dropna()
data_2b_filled.printSchema()

root
 |-- ncm: string (nullable = true)
 |-- id_pais_origem: string (nullable = true)
 |-- importador_municipio: string (nullable = true)
 |-- importador_uf: string (nullable = true)
 |-- urf: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- semestre: integer (nullable = true)


In [54]:
data_2b_filled = data_2b_filled.filter((data_2b_filled.urf != "") & 
                      (data_2b_filled.importador_uf != "") &
                      (data_2b_filled.importador_municipio != ""))

In [55]:
data_filled = data_2b_filled.join(historic_data, ["ncm", "id_pais_origem", "importador_municipio", "importador_uf", "urf", "ano", "semestre"], how='left')
data_filled.printSchema()

root
 |-- ncm: string (nullable = true)
 |-- id_pais_origem: string (nullable = true)
 |-- importador_municipio: string (nullable = true)
 |-- importador_uf: string (nullable = true)
 |-- urf: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- semestre: integer (nullable = true)
 |-- anomes: string (nullable = true)
 |-- avg_valor_item: double (nullable = true)
 |-- ano_semestre: long (nullable = true)
 |-- __index_level_0__: long (nullable = true)


In [56]:
data_filled.show(5)

+---------+--------------+--------------------+-------------+--------------------+----+--------+------+-----------------+------------+-----------------+
|      ncm|id_pais_origem|importador_municipio|importador_uf|                 urf| ano|semestre|anomes|   avg_valor_item|ano_semestre|__index_level_0__|
+---------+--------------+--------------------+-------------+--------------------+----+--------+------+-----------------+------------+-----------------+
|1.00191E7|        MÉXICO|                  RS|  PASSO FUNDO|AEROPORTO INTERNA...|2024|       2|  null|             null|        null|             null|
|1.00199E7|     ARGENTINA|       CAXIAS DO SUL|           RS|        IRF - ITAQUI|2023|       2|  null|             null|        null|             null|
|1.00199E7|     ARGENTINA|                  DF|     BRASILIA|     PORTO DE SANTOS|2024|       1|  null|             null|        null|             null|
|1.00199E7|     ARGENTINA|           FORTALEZA|           CE|     ALF - FORTALEZA|


## Ler os dados de linha de tendência;

In [57]:
tl_data = spark.read.parquet("../data/processed/trend_values/trend_lines.parquet")
tl_data.printSchema()
# tl_data = grouped_data[(grouped_data["importador_uf"] != "") & (grouped_data["importador_municipio"] != "")].copy()

root
 |-- ano: integer (nullable = true)
 |-- semestre: long (nullable = true)
 |-- ano_semestre: long (nullable = true)
 |-- avg_valor_item: double (nullable = true)
 |-- ncm: string (nullable = true)
 |-- id_pais_origem: string (nullable = true)


## Juntar histórico com gabarito futuro

In [58]:
data_trended = data_filled.join(tl_data, ["ncm", "id_pais_origem"], how='left').select(data_filled.ncm, 
                                                                                        data_filled.id_pais_origem,
                                                                                        data_filled.importador_municipio,
                                                                                        data_filled.importador_uf,
                                                                                        data_filled.urf,
                                                                                        data_filled.ano,
                                                                                        data_filled.semestre,
                                                                                        tl_data.avg_valor_item)

# Interpolar meses faltantes

In [60]:
data_trended.write.parquet("../data/interim/trended_values/", mode="overwrite")

In [61]:
import pyarrow.parquet as pq

table = pq.read_table("../data/interim/trended_values/")
df = table.to_pandas()

In [62]:
df.head()

,ncm,id_pais_origem,importador_municipio,importador_uf,urf,ano,semestre,avg_valor_item
0,1.00191E7,MÉXICO,RS,PASSO FUNDO,AEROPORTO INTERNACIONAL DE VIRACOPOS,2024,2,1.356255
1,1.00191E7,MÉXICO,RS,PASSO FUNDO,AEROPORTO INTERNACIONAL DE VIRACOPOS,2024,2,1.352692
2,1.00191E7,MÉXICO,RS,PASSO FUNDO,AEROPORTO INTERNACIONAL DE VIRACOPOS,2024,2,1.356255
3,1.00191E7,MÉXICO,RS,PASSO FUNDO,AEROPORTO INTERNACIONAL DE VIRACOPOS,2024,2,1.352692
4,1.00191E7,MÉXICO,RS,PASSO FUNDO,AEROPORTO INTERNACIONAL DE VIRACOPOS,2024,2,1.356255


In [63]:
df["avg_valor_item"] = df["avg_valor_item"].interpolate()

In [64]:
df.columns

Index(['ncm', 'id_pais_origem', 'importador_municipio', 'importador_uf', 'urf',
       'ano', 'semestre', 'avg_valor_item'],
      dtype='object')

In [ ]:
df_grouped = df.groupby(['ncm', 'id_pais_origem', 'importador_municipio', 'importador_uf', 'urf','ano', 'semestre'], \
                        as_index=False).mean('avg_valor_item')

In [ ]:
df_grouped["old_municipio"] = df_grouped["importador_municipio"]

In [ ]:
estados_br = ["AC","AL","AP","AM","BA","CE","ES","GO","MA","MT","MS","MG","PA","PB","PR","PE",
              "PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO","DF"]

print("Ajustando importador UF")
df_grouped["importador_uf_new"] = df_grouped.apply(lambda x: x["importador_uf"] if x["importador_uf"] in estados_br \
    else x["importador_municipio"], axis=1)

print("Ajustando importador municipio")
df_grouped["importador_municipio_new"] = df_grouped.apply(lambda x: x["old_municipio"] if x["old_municipio"] not in estados_br \
    else x["importador_uf"],axis=1)

In [ ]:
df_grouped.drop(columns=['importador_municipio', "importador_uf", "old_municipio"], inplace=True)

In [ ]:
df_grouped.rename(columns={"importador_municipio_new": "importador_municipio", 
                   "importador_uf_new": "importador_uf",
                    "id_pais_origem": "name_pt"}, inplace=True)

In [ ]:
df_grouped = df_grouped[df_grouped["avg_valor_item"] > 0]

In [ ]:
df_grouped.head()

In [ ]:
df_grouped.to_parquet("../data/processed/trend_line_and_historic_average_unity_price.parquet",index=False)

In [ ]:
import plotly.express as px

px.histogram(df_grouped, x='avg_valor_item')